# Projekat iz predmeta analiza socijalnih mreza


## Kvantitativna i kvalitativna Analiza stanja profesionalnog muškog tenisa u periodu od 2018. do 2020. godine


Ucitavanje podataka


In [1]:
import pandas as pd

dataPath = "data/atp_matches_2018.csv"

players_dataPath = "data/atp_players.csv"

player_atp_rankings_dataPath = "data/atp_rankings_10s.csv"

data = pd.read_csv(dataPath) 
#print(data.columns)
data.isna().any()

#players
players_data = pd.read_csv(players_dataPath)
#print(players_data)

#atp ranking
players_atp_rank_data = pd.read_csv(player_atp_rankings_dataPath)
players_atp_rank_data.isna().any()
#points null moze biti
#print(players_atp_rank_data)

ranking_date    False
rank            False
player          False
points           True
dtype: bool

In [2]:
#provera da li je match_num kolona koja moze da bude uzeta za id

print(data['match_num'].isnull().values.any())
print(data['match_num'].is_unique)

In [3]:
#dodati automatski generisani id kao dodatnu kolonu
data['id'] = data.index
print(data)

#sacuvati u novi fajl
import pickle

with open("data/data2018_cleaned.csv", 'wb') as file:
    pickle.dump(data, file)

In [4]:
import numpy as np

# Selekcija samo kolona 'match_num' i 'id' i grupisanje po match_num
data_grupisano = data[['match_num', 'id']].groupby('match_num')

# agg() radi agregaciju, u ovom slučaju po broju elemenata u grupi
data_agregirano = data_grupisano['id'].agg(np.size)
print(type(data_agregirano))
print(data_agregirano)

In [5]:
data_nonunique = data_agregirano[data_agregirano>1]
print(data_nonunique)

In [6]:
#naci max i index max broja ponovljenih
trazeni = data_nonunique[data_nonunique == data_nonunique.max()]
print(trazeni)
target_match_num = trazeni.keys()[0]
print(target_match_num)

match_num
1    67
2    67
4    67
Name: id, dtype: int64
1


In [9]:
#ispisati sve meceve kao match_num imaju odredjeni max
data_target = data[data['match_num'] == target_match_num]
lista_labela = ['tourney_name', 'winner_id', 'loser_id', 'winner_rank', 'loser_rank']
print(data_target[lista_labela])

#ispisi sve ciji je ili winner_rank null ili loser_rank null
data_winOrlos_is_null = data_target[(data_target['winner_rank'].isnull()) | (data_target['loser_rank'].isnull())]
print(data_winOrlos_is_null)

#istraga o igracima ciji je winner_rank u tabeli matches_2018 null ili loser_rank u tabeli matches_2018 null
print(players_data['player_id'].equals(data_winOrlos_is_null['winner_id']))
players_info_rang_null = players_data.loc[(players_data['player_id'].isin(data_winOrlos_is_null['winner_id'])) | (players_data['player_id'].isin(data_winOrlos_is_null['loser_id']))]
print(players_data.loc[(players_data['player_id'].isin(data_winOrlos_is_null['winner_id'])) | (players_data['player_id'].isin(data_winOrlos_is_null['loser_id']))])

#postoje neki koji uopste nemaju upisanu ruku i datum rodjenja, datum rodjenja format double iako u opisu drugaciji format

                     tourney_name  winner_id  loser_id  winner_rank  \
266   Davis Cup G1 R1: AUT vs BLR     105589    125802         98.0   
270   Davis Cup G1 R1: CHI vs ECU     111797    105111         95.0   
273   Davis Cup G1 R1: CHN vs NZL     104907    105842        308.0   
276   Davis Cup G1 R1: COL vs BAR     123755    104656        292.0   
279   Davis Cup G1 R1: DOM vs BRA     105497    106252        284.0   
...                           ...        ...       ...          ...   
2425  Davis Cup WG SF: FRA vs ESP     105332    105807         54.0   
2429  Davis Cup G2 R3: ROU vs POL     105657    111794         81.0   
2433  Davis Cup G2 R3: THA vs LBN     200273    106397        643.0   
2437  Davis Cup G2 R3: URU vs MEX     104655    106250         68.0   
2886   Davis Cup WG F: FRA vs CRO     106432    104871         12.0   

      loser_rank  
266        190.0  
270        597.0  
273        253.0  
276          NaN  
279        365.0  
...          ...  
2425        21

In [10]:
#players_info_rang_null 
#players_atp_rank_data
columns_name_players_atp_rank = ['ranking_date', 'rank', 'player', 'points']
print(players_atp_rank_data[columns_name_players_atp_rank])
print(players_atp_rank_data[columns_name_players_atp_rank].groupby('player').mean())

        ranking_date  rank  player   points
0           20100104     1  103819  10550.0
1           20100104     2  104745   9205.0
2           20100104     3  104925   8310.0
3           20100104     4  104918   7030.0
4           20100104     5  105223   6785.0
...              ...   ...     ...      ...
916291      20191230  1922  134833      1.0
916292      20191230  1922  144856      1.0
916293      20191230  1922  202326      1.0
916294      20191230  1926  207307      1.0
916295      20191230  1927  208186      1.0

[916296 rows x 4 columns]
        ranking_date         rank       points
player                                        
100644  2.015885e+07   247.552326  2102.796512
101389  2.014282e+07  1724.404255     1.000000
101397  2.011134e+07  1204.478261     3.000000
101404  2.011255e+07  1031.626374     6.571429
101549  2.010697e+07  1468.750000     1.000000
...              ...          ...          ...
209397  2.019123e+07  1581.000000     1.000000
209398  2.019122e+07  

In [11]:
import networkx as nx

list_winner = set(data['winner_id'].apply(str).unique())
print(list_winner)
list_loser = set(data['loser_id'].apply(str).unique())
print(list_loser)

skup_id = list_winner.union(list_loser)
print(f"Postoji { len(skup_id) } različitih igrača") # formatiranje ispisa pomoću fstringa

print(data.columns)

In [12]:
import numpy as np
import matplotlib.pyplot as plt

data_reduced = data[['winner_id', 'winner_name', 'winner_ioc', 'loser_id', 'loser_name', 'loser_ioc', 'id']]

G = nx.Graph() # pravi praznu mrežu
G.add_nodes_from(skup_id) # dodaje čvorove čije su labele u "skup_id"

for _, winner_id, winner_name, winner_ioc, loser_id, loser_name, loser_ioc, _ in data_reduced.itertuples():
    winner_id_string = str(winner_id)
    loser_id_string = str(loser_id)
    if (winner_id_string, loser_id_string) in G.edges:
        G.edges[winner_id_string, loser_id_string]['weight'] += 1
    else:
        G.add_edge(winner_id_string, loser_id_string, weight=1)
        winner_attrs = {winner_id_string: {'name': winner_name, 'country': winner_ioc}}
        loser_attrs = {loser_id_string: {'name': loser_name, 'country': loser_ioc}}
        nx.set_node_attributes(G, winner_attrs)
        nx.set_node_attributes(G, loser_attrs)
        


In [13]:
print(data.dtypes)
print(G.edges.data("weight"))
print(nx.get_node_attributes(G, "name"))
print(nx.get_node_attributes(G, "country"))
output_path = "models/undirected_graph_2018.gml"

nx.write_gml(G, output_path)


1.pitanje Koliki je prosecan broj tenisera(saigraca) po svakom teniseru? 
gephi->ucitala graf, Statistics -> Network Overview -> Average Degree

<img src="img/1.pitanje_average_degree" width=500 height=500 />

In [14]:
#2.pitanje Koji teniseri su se susretali sa najvecim brojem drugih tenisera
my_degrees = dict(G.degree()) #dictionary key:value
degree_values = [v for k, v in G.degree()]

print(max(my_degrees.items(), key= lambda x: x[1])) #max u tom dict 

print(nx.get_node_attributes(G, 'name')['104926']) #atribut name za cvor ciji je index rezultat prethodnog upita

('104926', 61)
Fabio Fognini


In [15]:
#3.pitanje Koji teniseri su ucestvovali na najvecem broju (razlicih) turnira?
winner_tourney_columns = ['winner_id', 'tourney_id']
loser_tourney_columns = ['loser_id', 'tourney_id']
dataWinner_tourney = data[winner_tourney_columns].drop_duplicates().groupby('winner_id').count()
print(dataWinner_tourney)
dataLoser_tourney = data[loser_tourney_columns].drop_duplicates().groupby('loser_id').count()
print(dataLoser_tourney)

print(nx.get_node_attributes(G, 'name')['100644'])
print(nx.get_node_attributes(G, 'name')['103333'])

#check???


           tourney_id
winner_id            
100644             20
103333              7
103499              2
103529              1
103565              1
...               ...
202358              1
207139              1
207415              1
208028              1
208029              1

[282 rows x 1 columns]
          tourney_id
loser_id            
100644            19
103333            16
103499             1
103529             1
103565             2
...              ...
207738             1
207836             1
207971             1
207972             1
207973             1

[393 rows x 1 columns]
Alexander Zverev
Ivo Karlovic


In [16]:
#4.pitanje Koji teniseri su dobri kandidati za predstavnike profesionalnih tenisera? Da li se i sta se menja ukoliko se umesto jednog bira skup od nekoliko predstavnika? Dobar predstavnik ili predstavnici bi trebalo da budu u kontaktu sa sto vecim skupom tenisera

#proveriti ovaj 2.deo na sta se odnosi????
#mere centralnosti koristim Degree centrality, vise u kontaktu sa drugim teniserima teze da budu predstavnici
import operator
dictionary = dict(nx.degree_centrality(G))
print(sorted(dictionary.items(), key=operator.itemgetter(1), reverse=True)[:5])

[('104926', 0.14593301435406697), ('106233', 0.13157894736842105), ('126774', 0.12679425837320574), ('100644', 0.12440191387559808), ('104755', 0.12200956937799043)]


In [17]:
print(nx.get_node_attributes(G, 'name')['104926'])
print(nx.get_node_attributes(G, 'name')['106233'])
print(nx.get_node_attributes(G, 'name')['126774'])
print(nx.get_node_attributes(G, 'name')['100644'])
print(nx.get_node_attributes(G, 'name')['104755'])

Fabio Fognini
Dominic Thiem
Stefanos Tsitsipas
Alexander Zverev
Richard Gasquet


Prvo kreiramo masku koja ce da selektuje sve igrace koji su u skupu igraca koji su igrali mec u 2018
Potom dobijen rezultat grupisemo po drzavi i sortiramo zatim
Najaktivniji su iz SAD, Francuske, Spanije, Nemacke, Italije
Srbija na 9 mestu sa Rusijom i Kanadom sa 7 aktivnih igraca

In [18]:
maska_active_players = players_data['player_id'].isin(skup_id) 
active_players_2018 = players_data[maska_active_players]
data_grupisano = active_players_2018[['player_id', 'country_code']].groupby('country_code')
data_agregirano = data_grupisano['country_code'].agg(np.size)

print(data_agregirano.sort_values(ascending=False).head(20))

country_code
USA    34
FRA    25
ESP    24
GER    18
ITA    17
ARG    15
AUS    14
AUT     8
RUS     7
CAN     7
SRB     7
GBR     6
CZE     6
CRO     6
ROU     6
SVK     6
SUI     6
BRA     6
TPE     6
IND     6
Name: country_code, dtype: int64


Analiza komuna

implementiracemo bottom-up pristup -> Louvain metod

In [27]:
import sys
!{sys.executable} -m pip install python-louvain
import community as community_louvain
import matplotlib.pyplot as plt
import matplotlib.cm as cm
partition = community_louvain.best_partition(G)

print(partition)

#vizuelno
pos = nx.spring_layout(G)
cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=200, cmap=cmap, node_color=list(partition.values()))
nx.draw_networkx_edges(G, pos, alpha=1)
plt.show()

In [20]:
player_id, clustering_coef = zip(*nx.clustering(G, weight = "weight").items())

nenula = [(id_ig, cc)  for id_ig, cc in zip(player_id, clustering_coef) if cc > 0]

df = pd.DataFrame(nenula, columns = ["id", "cc"])
df.sort_values('cc', inplace = True)

max_lokalni_stepen_klasterisanja = max(clustering_coef)

prosecni_stepen_klasterisanja = nx.average_clustering(G)

print(f"Max lokalni cc: {max_lokalni_stepen_klasterisanja}")
print(f"Prosečan cc: {prosecni_stepen_klasterisanja}")
print("Lokalni stepeni klasterisanja koji nisu nula:")
print(df)

Max lokalni cc: 0.20000000000000004
Prosečan cc: 0.12227548001603938
Lokalni stepeni klasterisanja koji nisu nula:
         id        cc
85   105967  0.003030
17   104122  0.013333
66   106071  0.013767
63   106216  0.014286
76   105132  0.015152
..      ...       ...
61   132482  0.192659
59   200175  0.200000
34   104672  0.200000
70   200218  0.200000
162  126523  0.200000

[204 rows x 2 columns]


Asortativnost:
Homofilija - tendencija ljudi da stvaraju veze sa sebi slicnim ljudima
U analizi socijalnih mreze homofilija u formi asortativnosti - cvorovi imaju tendenciju da se povezuju sa cvorovima slicnog stepena

In [19]:
# asortativnost na osnovu netežinskog stepena čvora
r1 = nx.degree_assortativity_coefficient(G)
print(f"Koeficijent asortativnosti na osnovu netežinskog stepena čvora: {r1}")

# asortativnost na osnovu težinskog stepena čvora
r2 = nx.degree_assortativity_coefficient(G, weight='weight')
print(f"Koeficijent asortativnosti na osnovu težinskog stepena čvora: {r2}")

Koeficijent asortativnosti na osnovu netežinskog stepena čvora: 0.21868975600564688
Koeficijent asortativnosti na osnovu težinskog stepena čvora: 0.22945507317379313
